In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# Author: sjangbahadur@

In [ ]:
!pip install --upgrade google-genai
!pip install pandas
!pip install google-cloud-storage

In [ ]:
import sys
from IPython.display import Image, display

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
import pandas as pd
from google.cloud import storage
import os

# Specify the path to your XLSX file
# If the file is not in the current environment, please upload it to the Colab environment first.
# Example: file_path = 'your_file_name.xlsx'
file_path = "Bulk Marketing Generation.xlsx" # @param {type:"string"}

# Check if the file path is provided
if file_path:
    # Load the XLSX file into a pandas DataFrame
    df = pd.read_excel(file_path)

    # Display the first few rows of the DataFrame to verify
    print("DataFrame loaded successfully. First 5 rows:")
    print(df.head())
else:
    print("Please provide the path to your XLSX file in the 'file_path' variable.")

In [ ]:
import re
import os
import json
def format_json_response(response_text, folder_path):
  print(response_text)
  response_text_temp = re.sub(r"json", "", response_text)
  response_text_temp = re.sub(r"```", "", response_text_temp)
  json_response = json.loads(response_text_temp)
  filename = os.path.join(folder_path ,"output.json")
  print(filename)
  # Open the file in write mode ('w') and use json.dump() to write the data
  with open(filename, 'w') as file:
     json.dump(json_response, file, indent=4) # 'indent=4' for pretty-printing with 4 spaces

  print(f"JSON data successfully written to '{filename}'")
  return json_response

def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the Google Cloud Storage bucket."""
    # Initialize a client
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    gcs_path = f"gs://{bucket_name}/{destination_blob_name}"
    print(f"File {source_file_name} uploaded to {gcs_path}")
    return gcs_path

In [ ]:
project_id = "<PROJECT_ID>" # @param {type:"string"}
region = "global" # @param {type:"string"}
GCS_BUCKET_NAME = "<GCS_BUCKET_NAME>" # @param {type:"string"}
VEO_MODEL_ID = "veo-3.1-generate-preview" # @param {type:"string"}
GEMINI_MODEL = "gemini-3-pro-preview" # @param {type:"string"}
IMAGE_MODEL = "gemini-3-pro-image-preview" # @param {type:"string"}

In [ ]:
from google import genai
from google.genai import types
import base64
import os
import requests
import io
from io import BytesIO

from IPython.display import Image, Markdown, display
from PIL import Image as PIL_Image
from google import genai
from google.genai.types import GenerateContentConfig, Part
import matplotlib.image as img
import matplotlib.pyplot as plt
import requests
import time

client = genai.Client(
      vertexai=True,
      project=project_id,
      location=region,
  )

def generate_marketing_content(product_name, specs, product_image_bytes):

  msg1_text1 = types.Part.from_text(text=f"""SYSTEM:```You are an award-winning Advertising Director and Creative Copywriter specializing in high-conversion short-form video content.
  Your expertise lies in creating 30-second commercials that utilize realistic AI avatars to build trust and explain complex products simply.
  Your goal is to take a product name, its specifications, and a product image URL, and transform them into a structured JSON payload that serves as a programmatic blueprint for video generation.
  You prioritize realism, clarity, and engagement.```
  INSTRUCTION:```Based on the provided {product_name}, {specs}, and product image given below, generate a detailed 30+ second video production script in strict JSON format.
  The video must be broken down into sequential scenes (approx 5-6 scenes, totaling 30+ seconds).
  Constraints:
  1. JSON Only: Do not include conversational text outside the JSON object.
  2. Timing: The sum of all `duration_seconds` must equal 30.
  3. Logic: Ensure smooth transitions between scenes.
  4. Image Usage: You must explicitly describe how the provided product image is visually integrated into every scene.
  5. Scripting: Translate technical {{specs}} into user-centric benefits.```
  Product_image:""")
  msg1_text2 = types.Part.from_text(text="""Output Format:```Return a single JSON object with the following structure:
  {
     "video_title": "string",
     "total_duration": 30,
     "avatar_profile": {
       "gender": "string",
       "age_range": "string",
       "attire": "string (professional yet approachable)",
       "tone_of_voice": "string",
       "visual_description": "Detailed physical description of the avatar to ensure consistency across scenes. Make background white color."
      },
      "scenes": [
        {
        "scene_number": 1,
        "duration_seconds": 8,
        "scene_type": "e.g., The Hook, The Problem, The Solution",
        "visual_background": "Detailed description of the environment.",
        "avatar_action": "Specific gestures or movements.",
        "product_visual_integration": "How the product_image_url is displayed (e.g., 'Holographic overlay next to avatar', 'Picture-in-Picture top right', 'Avatar holding a tablet displaying the image').",
        "script_dialogue": "The crisp exact spoken words which fits within max 8 sec."
        }
      ]
    }```""")
  image1 = types.Part.from_bytes(data=product_image_bytes,
                           mime_type="image/*")
  contents = [
    types.Content(
      role="user",
      parts=[
        msg1_text1,
        image1,
        msg1_text2
      ]
    ),
  ]

  generate_content_config = types.GenerateContentConfig(
  )

  response = client.models.generate_content(
    model = GEMINI_MODEL,
    contents = contents,
    config = generate_content_config,
  )
  #print(response.text)
  return response.text

In [ ]:
def read_file(file_name):
  file_content = ""
  with open(file_name, "rb") as f:
    file_content = f.read()
  return file_content

def generate_story_board_images(text_prompt, storyboard_file_name, avatar_image=None, product_image=None):
  parts = []
  if avatar_image:
    msg1_image1 = types.Part.from_uri(
        file_uri=avatar_image,
        mime_type="image/*",
    )
    parts.append(msg1_image1)
  if product_image:
    msg1_image2 = types.Part.from_uri(
        file_uri=product_image,
        mime_type="image/*",
    )
    parts.append(msg1_image2)
  parts.append(types.Part.from_text(text=text_prompt))

  contents = [
    types.Content(
      role="user",
      parts=parts
    ),
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 32768,
    response_modalities = ["IMAGE"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    image_config=types.ImageConfig(
      aspect_ratio="9:16",
      image_size="1K",
      output_mime_type="image/png",
    ),
  )

  response = client.models.generate_content(
    model=IMAGE_MODEL,
    contents=contents,
    config=generate_content_config
  )

  for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    if part.inline_data:
        with open(storyboard_file_name, 'wb') as f:
            f.write(part.inline_data.data)
        print(f"Image saved to {storyboard_file_name}")
        display(Image(data=part.inline_data.data, width=500))

In [ ]:
def storyboard_quality_check(reference_avatar_image, reference_product_image, storyboard_image):
  msg1_text1 = types.Part.from_text(text="""SYSTEM:```You are an expert Ad Video Director and Visual Continuity Specialist with a keen eye for detail. Your role is to perform rigorous quality assurance on video storyboards to ensure strict consistency with brand assets. You have an exceptional ability to detect subtle discrepancies in facial features, clothing details, product packaging, logos, and color grading. You evaluate visual inputs objectively and provide structured, data-driven feedback in JSON format. Your goal is to ensure that the character (avatar) and the merchandise (product) in the storyboard are identical to their respective references.```
INSTRUCTION:```Analyze the provided input images:
(1) Reference Avatar
(2) Reference Product and
(3) Storyboard Image.
Perform a comparative analysis to validate the visual consistency.
### STEPS:
1. Avatar Analysis:
	- Compare the character in the Storyboard Image against the Reference Avatar.
	- Scrutinize facial features, hair style/color, age, gender, ethnicity, and clothing style.
	- Determine if the identity is preserved or if there are hallucinations/distortions.
2. Product Analysis:
	- Compare the object/product in the Storyboard Image against the Reference Product.
	- Check for logo accuracy, spelling of text, color codes, shape, and packaging details.
	- Ensure the product has not been morphed or replaced.
3. Scoring & Reasoning:
	- Assign a consistency score from 0 to 100 for both the Avatar and the Product (where 100 is a pixel-perfect conceptual match and 0 is completely unrecognizable).
	- Provide a concise, specific reason for the score, highlighting exactly what matched or what failed (e.g., 'Face shape matches, but eye color is wrong' or 'Logo text is misspelled').```
REFERENCE_AVATAR_IMAGE:""")
  msg1_image1 = types.Part.from_bytes(
      data=read_file(reference_avatar_image),
      mime_type="image/*",
  )
  msg1_image2 = types.Part.from_bytes(
      data=read_file(reference_product_image),
      mime_type="image/*",
  )
  msg1_image3 = types.Part.from_bytes(
      data=read_file(storyboard_image),
      mime_type="image/*",
  )
  msg1_text2 = types.Part.from_text(text="""OUTPUT_FORMAT:
You must return ONLY a raw JSON object with no markdown formatting or additional text. Use the following schema:
```{
	"avatar_validation": {
		"score": <integer_0_to_100>,
		"reason": "<detailed_explanation_string>"
	},
	"product_validation": {
		"score": <integer_0_to_100>,
		"reason": "<detailed_explanation_string>"
	}
}```""")

  contents = [
    types.Content(
      role="user",
      parts=[
        msg1_text1,
        msg1_image1,
        types.Part.from_text(text="""REFERENCE_PRODUCT_IMAGE:"""),
        msg1_image2,
        types.Part.from_text(text="""STORYBOARD_IMAGE:"""),
        msg1_image3,
        msg1_text2
      ]
    ),
  ]
  tools = [
    types.Tool(google_search=types.GoogleSearch()),
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 65535,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    tools = tools,
    thinking_config=types.ThinkingConfig(
      thinking_level="HIGH",
    ),
  )

  response = client.models.generate_content(
    model = GEMINI_MODEL,
    contents = contents,
    config = generate_content_config,
  )
  #print(response.text)
  return response.text

In [ ]:
def generate_video_scenes(prompt_text, video_file_path, reference_image=None):
  # Optional parameters
  negative_prompt = "ugly, low quality"
  aspect_ratio = "9:16"
  resolution = "1080p"
  generate_audio = True
  duration_seconds = 8
  number_of_videos = 4

  # Loading the image
  if reference_image:
    #im = PIL_Image.open(reference_image)
    # converting the image to bytes
    #image_bytes_io = io.BytesIO()
    #im.save(image_bytes_io, format=im.format)
    #image_bytes = image_bytes_io.getvalue()

    operation = client.models.generate_videos(
        model=VEO_MODEL_ID,
        prompt=prompt_text,
        image=types.Image(
          gcs_uri=reference_image,
          mime_type="image/*",
        ),
        config=types.GenerateVideosConfig(
          aspect_ratio=aspect_ratio,
          resolution=resolution,
          number_of_videos=number_of_videos,
          duration_seconds=duration_seconds,
          negative_prompt=negative_prompt,
          generate_audio=generate_audio,
          output_gcs_uri=video_file_path,
        ),
    )
  else:
    operation = client.models.generate_videos(
        model=VEO_MODEL_ID,
        prompt=prompt_text,
        config=types.GenerateVideosConfig(
          aspect_ratio=aspect_ratio,
          resolution=resolution,
          number_of_videos=number_of_videos,
          duration_seconds=duration_seconds,
          negative_prompt=negative_prompt,
          generate_audio=generate_audio,
          output_gcs_uri=video_file_path,
        ),
    )

  while not operation.done:
    time.sleep(15)
    operation = client.operations.get(operation)
    print(operation)

  if operation.response:
    print(operation.result.generated_videos[0].video.uri)
    return operation.result.generated_videos
  else:
    return None

  # Waiting for the video(s) to be generated
  #while not operation.done:
  #    time.sleep(20)
  #    operation = client.operations.get(operation)
  #    print(operation)

  #print(operation.result.generated_videos)

  #for n, generated_video in enumerate(operation.result.generated_videos):
  #  client.files.download(file=generated_video.video)
  #  generated_video.video.save(f'{video_file_path}/videos_{n}.mp4') # Saves the video(s)
  #  display(generated_video.video.show()) # Displays the video(s) in a notebook

In [ ]:
def video_quality_check(video_gcs_path, reference_image_gcs_path):
  msg1_text1 = types.Part.from_text(text="""SYSTEM:```You are a veteran Film Director and VFX Quality Control Supervisor with over 20 years of experience in cinematic workflows and post-production. You possess an acute eye for detail, capable of spotting sub-pixel anomalies, temporal inconsistencies, and fidelity issues. Your task is to critique AI-generated or edited video content against reference asset with a rigorous standard for photorealism, continuity, and technical integrity. You communicate using technical cinematic terminology.```
INSTRUCTION:```Analyze the provided video input in relation to the provided reference Image''. You must conduct a frame-by-frame analysis to detect any degradation in quality or logical fallacies.
Evaluate the video based on the following five specific dimensions:
1. Technical Distortion: Check for compression artifacts, screen tearing, aliasing, warping, or glitching.
2. Cinematic Imperfections: Analyze lighting flickers, unnatural motion blur, grain inconsistencies, or color grading shifts that break the cinematic look.
3. Avatar Consistency: Compare the video subject against the reference Image. Check for facial feature consistency, and anatomical correctness during movement.
4. Product Consistency: Compare the object in the video against the Reference Image. Check for logo deformation, texture warping, scale issues, or color shifting.
Scoring Criteria:
- Provide a 'Quality Score' from 0 to 10 for each category (10 being perfect/flawless, 0 being unusable).
Make sure to double check the reference image and video frames before generating the results.```
VIDEO:""")
  msg1_video1 = types.Part.from_uri(
      file_uri=video_gcs_path,
      mime_type="video/mp4",
      media_resolution={"level": "media_resolution_high"}
  )
  msg1_image1 = types.Part.from_uri(
      file_uri=reference_image_gcs_path,
      mime_type="image/*",
      media_resolution={"level": "media_resolution_high"}
  )
  msg1_text2 = types.Part.from_text(text="""Output Format:```
- Your response must be strictly in valid JSON format.
- Do not include markdown formatting (like ```json) or conversational text outside the JSON object.
- Use the following schema:
{
	"analysis_report": {
		"technical_distortion": {
			"score": <0-10>,
			"reasoning": "Detailed technical explanation of artifacts observed..."
		},
		"cinematic_imperfections": {
			"score": <0-10>,
			"reasoning": "Analysis of lighting, shutter angle, camera movements and grain..."
		},
		"avatar_consistency": {
			"score": <0-10>,
			"reasoning": "Comparison of facial feature consistency against reference..."
		},
		"product_consistency": {
			"score": <0-10>,
			"reasoning": "Assessment of brand asset fidelity and object geometry..."
		},
		"overall_verdict": "A summary statement regarding the video's usability for professional broadcast."
	}
}```""")

  contents = [
    types.Content(
      role="user",
      parts=[
        msg1_text1,
        msg1_video1,
        types.Part.from_text(text="""REFERENCE_IMAGE:"""),
        msg1_image1,
        msg1_text2
      ]
    ),
  ]
  tools = [
    types.Tool(google_search=types.GoogleSearch()),
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 65535,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    tools = tools,
    thinking_config=types.ThinkingConfig(
      thinking_level="HIGH",
    ),
  )

  for chunk in client.models.generate_content_stream(
    model = GEMINI_MODEL,
    contents = contents,
    config = generate_content_config,
    ):
    if not chunk.candidates or not chunk.candidates[0].content or not chunk.candidates[0].content.parts:
        continue
    print(chunk.text, end="")


In [ ]:
# Define the folder name for storing product images
image_folder = "product_images"

# Create the folder if it doesn't exist
if not os.path.exists(image_folder):
    os.makedirs(image_folder)
    print(f"Created folder: {image_folder}/")

product_data = []
for index, row in df.iterrows():
    product_name = row['Product Name']
    specifications = row['Specifications']
    image_urls = row['Public Image URL']
    first_image_url = image_urls.split(',')[0].strip() if isinstance(image_urls, str) else image_urls

    product_image_bytes = None
    # Construct the full path for the local image file for the product
    folder_path = os.path.join(image_folder, f"product_{index}")
    os.makedirs(folder_path, exist_ok=True)
    product_local_image_path = os.path.join(folder_path, f"product_image_{index}.jpeg") # Original product image saved as jpeg
    print(product_local_image_path)
    if first_image_url:
        try:
            response = requests.get(first_image_url, stream=True)
            response.raise_for_status() # Raise an exception for HTTP errors
            product_image_bytes = io.BytesIO(response.content).getvalue()
            with open(product_local_image_path, 'wb') as f:
                f.write(product_image_bytes)
            print(f"Downloaded image for '{product_name}' from '{first_image_url}' and saved to '{product_local_image_path}'")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading image for '{product_name}' from '{first_image_url}': {e}")
            product_image_bytes = None # Ensure it's None if download fails

    combined_string = f"Product Name: {product_name}, Specifications: {specifications}"

    response_json = None
    if product_image_bytes:
        try:
            print(f"\n--- Generating marketing content for {product_name} ---")
            response_json = format_json_response(generate_marketing_content(product_name, specifications, product_image_bytes), folder_path)

            if response_json:
                # --- New Steps Start Here ---

                # 1. Generate unique avatar image
                print(f"\n--- Generating avatar image for {product_name} ---")
                avatar_profile = response_json['avatar_profile']
                avatar_prompt = json.dumps(avatar_profile)
                avatar_image_filename = os.path.join(folder_path, f"avatar_{index}.png") # Generated image is png
                generate_story_board_images(avatar_prompt, avatar_image_filename)

                gcs_avatar_image_destination_blob = f"bajaj_marketing_workflow/product_{index}/{os.path.basename(avatar_image_filename)}"
                gcs_avatar_image_path = upload_to_gcs(GCS_BUCKET_NAME, avatar_image_filename, gcs_avatar_image_destination_blob)
                print(gcs_avatar_image_path)

                gcs_product_image_destination_blob = f"bajaj_marketing_workflow/product_{index}/{os.path.basename(product_local_image_path)}"
                gcs_product_image_path = upload_to_gcs(GCS_BUCKET_NAME, product_local_image_path, gcs_product_image_destination_blob)
                print(gcs_product_image_path)

                # 2. Generate storyboard images for each scene & 3. Perform storyboard quality checks
                print(f"\n--- Generating storyboard images and performing QC for {product_name} ---")
                for scene in response_json['scenes']:
                    scene_image_filename = os.path.join(folder_path, f"scene_{index}_{scene['scene_number']}.png") # Generated image is png
                    storyboard_prompt = f"Background: {scene['visual_background']}, avatar_action : {scene['avatar_action']}, product_integration: {scene['product_visual_integration']}."
                    generate_story_board_images(storyboard_prompt, scene_image_filename, avatar_image=gcs_avatar_image_path, product_image=gcs_product_image_path)

                    gcs_scene_image_destination_blob = f"bajaj_marketing_workflow/product_{index}/{os.path.basename(scene_image_filename)}"
                    gcs_scene_image_path = upload_to_gcs(GCS_BUCKET_NAME, scene_image_filename, gcs_scene_image_destination_blob)

                    # Perform storyboard quality check
                    print(f"--- Performing storyboard quality check for scene {scene['scene_number']} ---")
                    try:
                        # storyboard_quality_check returns a JSON string
                        storyboard_qc_result = storyboard_quality_check(avatar_image_filename, product_local_image_path, scene_image_filename)
                        print(f"Storyboard QC Result for scene {scene['scene_number']}:\n{storyboard_qc_result}")
                    except Exception as qc_e:
                        print(f"Error during storyboard quality check for scene {scene['scene_number']}: {qc_e}")


                    # 4. Generate a promotional video
                    print(f"\n--- Generating promotional video for {product_name} ---")
                    video_output_folder = os.path.join(folder_path, "generated_videos")
                    os.makedirs(video_output_folder, exist_ok=True)
                    video_prompt = f"scene_type: {scene['scene_type']}, tone_of_voice: {avatar_profile['tone_of_voice']}, avatar_action : {scene['avatar_action']}, visual_background: {scene['visual_background']}, product_visual_integration: {scene['product_visual_integration']}, Dialogue:{scene['script_dialogue']}"
                    gcs_video_destination_blob = f"gs://{GCS_BUCKET_NAME}/bajaj_marketing_workflow/product_{index}"
                    gcs_video_paths = generate_video_scenes(video_prompt, gcs_video_destination_blob, reference_image=gcs_scene_image_path)
                    # Assuming the generated video is named videos_0.mp4 inside the video_output_folder
                    #generated_video_local_path = os.path.join(video_output_folder, "videos_0.mp4")
                    for video_path in gcs_video_paths:
                      # 5. Upload the generated video and original product image to GCS
                      print(f"\n--- Uploaded video to GCS: {video_path.video.uri} ---")
                      #gcs_video_destination_blob = f"bajaj_marketing_workflow/product_{index}/{os.path.basename(generated_video_local_path)}"
                      #gcs_video_path = upload_to_gcs(GCS_BUCKET_NAME, generated_video_local_path, gcs_video_destination_blob)

                      # 6. Perform a quality check on the generated video
                      print(f"\n--- Performing video quality check for {product_name} ---")
                      try:
                          # video_quality_check function prints its output directly
                          video_quality_check(video_path.video.uri, gcs_scene_image_path)
                      except Exception as vid_qc_e:
                          print(f"Error during video quality check: {vid_qc_e}")

                # --- New Steps End Here ---

        except Exception as e:
            print(f"Error during processing for '{product_name}': {e}")
            import traceback
            traceback.print_exc() # Print full traceback for debugging
            print("This could be due to issues with API calls or file operations.")
    else:
        print(f"Skipping processing for '{product_name}' due to missing or failed image download.")

    product_data.append(combined_string)

print(f"Extracted {len(product_data)} product entries.")
